In [ ]:
from torch.utils.data import Dataset
from transformers import BartTokenizer
import torch
from transformers import AutoTokenizer
from transformers import AutoModel ,BartForConditionalGeneration
import pandas as pd
from transformers import Trainer
from transformers import TrainingArguments

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# try:
#     checkpoint = torch.load('/content/drive/MyDrive/bart/pytorch_model.bin')
#     print("Checkpoint loaded successfully.")
# except Exception as e:
#     print("Failed to load checkpoint:", e)

Failed to load checkpoint: PytorchStreamReader failed reading zip archive: failed finding central directory


In [ ]:
model_name = "/content/drive/MyDrive/bart"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device



device(type='cuda')

In [ ]:
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [ ]:
# ner_pipeline = pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True)

data = pd.read_excel(r'/content/drive/MyDrive/bart/MeQSum_ACL2019_BenAbacha_Demner-Fushman.xlsx')

In [ ]:

class CustomDataset(Dataset):

    def __init__(self, dataframe, source_col, target_col, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_col = source_col
        self.target_col = target_col
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        source = str(self.data[self.source_col][index])
        target = str(self.data[self.target_col][index])

        source = self.tokenizer.encode_plus(
            source,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt"
        )

        target = self.tokenizer.encode_plus(
            target,
            max_length=self.max_len,
            padding='max_length',
            return_tensors="pt"
        )

        return {
            "input_ids": source["input_ids"].flatten(),
            "attention_mask": source["attention_mask"].flatten(),
            "labels": target["input_ids"].flatten(),
        }

# inputs = tokenizer(source_texts, truncation=True, padding=True, max_length=512, return_tensors="tf")
# labels = tokenizer(target_texts, truncation=True, padding=True, max_length=128, return_tensors="tf")

dataset = CustomDataset(data, 'CHQ', 'Summary', tokenizer, 512)

In [ ]:

training_args = torch.load("/content/drive/MyDrive/bart/training_args.bin")

FileNotFoundError: ignored

In [ ]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.5 MB/s eta 0:00:00


In [ ]:
!pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.6/188.6 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.8/218.8 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=e54fa62a2c7a881965be51ad9a77b47fff01ada7bb12f4bda7c7d389f7b7b846
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
!pip install transformers==4.24.0
!pip install torch==1.12.1
!pip install datasets==2.6.1
!pip install tokenizers==0.13.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.0
    Uninstalling transformers-4.33.0:
      Successfully uninstalled transformers-4.33.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.3/776.3 MB 2.4 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1+cu118
    Uninstalling torch-2.0.1+cu118:
      Successfully uninstalled torch-2.0.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.12.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.12.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.12.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.12.1 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 441.9/441.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 13.0 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3


In [ ]:

training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/bart_results',
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    warmup_steps=100,
    weight_decay=0.02,
    logging_dir='./logs',
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from accelerate import Accelerator
model = model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 3000
  Number of trainable parameters = 139480320
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,1.848200
1000,0.066200
1500,0.040800
2000,0.040200
2500,0.025200
3000,0.026500


Saving model checkpoint to /content/drive/MyDrive/bart_results/checkpoint-500
Configuration saved in /content/drive/MyDrive/bart_results/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/bart_results/checkpoint-500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/bart_results/checkpoint-1000
Configuration saved in /content/drive/MyDrive/bart_results/checkpoint-1000/config.json
Model weights saved in /content/drive/MyDrive/bart_results/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/bart_results/checkpoint-1500
Configuration saved in /content/drive/MyDrive/bart_results/checkpoint-1500/config.json
Model weights saved in /content/drive/MyDrive/bart_results/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/bart_results/checkpoint-2000
Configuration saved in /content/drive/MyDrive/bart_results/checkpoint-2000/config.json
Model weights saved in /content/drive/MyDrive/bart_results/c

TrainOutput(global_step=3000, training_loss=0.3411953773498535, metrics={'train_runtime': 941.2751, 'train_samples_per_second': 3.187, 'train_steps_per_second': 3.187, 'total_flos': 914604687360000.0, 'train_loss': 0.3411953773498535, 'epoch': 3.0})

In [ ]:
source_texts = data.iloc[:, 1].tolist()
test_list = source_texts[:100]

In [ ]:
model_path = '/content/drive/MyDrive/bart_results/checkpoint-2500'

model = BartForConditionalGeneration.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_name)
model = model.to(device)

loading configuration file /content/drive/MyDrive/bart_results/checkpoint-2500/config.json
Model config BartConfig {
  "_name_or_path": "/content/drive/MyDrive/bart",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "init_std": 0.02,
  "is_encoder_deco

In [ ]:
torch.cuda.empty_cache()
input_text = r"SUBJECT: CPTII Deficiency MESSAGE: My sister in Texas was just diagnosed with CPTII deficiency today.  We do not yet know which variant.  I was told a year ago following her near death from rhabdo (CK's 150,000+), ATN, RF, respiratory compromise, to be tested for a FA metabolic error.  Until today, I really had no idea which test to request.  Her muscle bx was performed a few months ago as she was still ill last year.  Another recurrence of rhado this year prompted the bx (she is retired military).  I don't know which branch of medicine to contact for genetic testing. From  the NIH website, muscle bx is the most definitive.  I also found a phone number for the Genetics and Rare Diseases Information Center.  Thank you for some direction."

# Tokenize input text
inputs = tokenizer(input_text, return_tensors='pt')

# Generate prediction
inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}
outputs = model.generate(**inputs)

# Decode the prediction
prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(prediction)

Where can I get genetic testing for CPTII deficiency?


/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:1359: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
inputs = tokenizer(source_texts, return_tensors='pt', padding=True, truncation=True)
inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}
outputs = model.generate(**inputs)
predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/autograd/grad_mode.py:27 in decorate_context       │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:1339 in generate        │
│                                                                                                  │
│   1336 │   │   if self.config.is_encoder_decoder and "encoder_outputs" not in model_kwargs:      │
│   1337 │   │   │   # if model is encoder decoder encoder_outputs are created                     │
│   1338 │   │   │   # and added to `model_kwargs`                                                 │
│ ❱ 1339 │   │   │   model_kwargs = self._prepare_encoder_decoder_kwargs_for_generation(           │
│   1340 │   │   │   │   inputs_tensor, model_kwargs, model_input_name                             │
│   1341 │   │   │   )                                                                             │
│   1342                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:583 in                  │
│ _prepare_encoder_decoder_kwargs_for_generation                                                   │
│                                                                                                  │
│    580 │   │   model_input_name = model_input_name if model_input_name is not None else self.ma  │
│    581 │   │   encoder_kwargs["return_dict"] = True                                              │
│    582 │   │   encoder_kwargs[model_input_name] = inputs_tensor                                  │
│ ❱  583 │   │   model_kwargs["encoder_outputs"]: ModelOutput = encoder(**encoder_kwargs)          │
│    584 │   │                                                                                     │
│    585 │   │   return model_kwargs                                                               │
│    586                                                                                           │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1130 in _call_impl            │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)   

In [ ]:
for i, prediction in enumerate(predictions):
    print(f"Text {i+1}: {prediction}")